In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import RandomForestRegressor

spark = SparkSession.builder.appName('treecodealong').getOrCreate()

In [2]:
data = spark.read.format('csv').load('/FileStore/tables/Machine_learning/College.csv', inferschema = True, header = True)

In [3]:
#data.printSchema()
#data.head(1)
data.columns

In [4]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol = 'features')

In [5]:
output = assembler.transform(data)

In [6]:
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [7]:
output_fixed = indexer.fit(output).transform(output)

In [8]:
output_fixed.printSchema()

In [9]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [10]:
train, test = final_data.randomSplit([0.7,0.3])

In [11]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features')
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [12]:
dtc_model = dtc.fit(train)
rfc_model =rfc.fit(train)
gbt_model = gbt.fit(train)

In [13]:
dtc_preds = dtc_model.transform(test)
rfc_preds = rfc_model.transform(test)
gbt_preds = gbt_model.transform(test)

In [14]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [15]:
#print('DTC')
#print(my_binary_eval.evaluate(dtc_preds))
print('rfc')
print(my_binary_eval.evaluate(rfc_preds))

In [16]:
rfc_preds.printSchema()

In [17]:
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction')

In [18]:
print('gbt')
print(my_binary_eval2.evaluate(gbt_preds))

In [19]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [20]:
rfc_acc = acc_eval.evaluate(rfc_preds)
rfc_acc